# Download Sentence-BERT para uso Offline

**Este notebook baixa o Sentence-BERT multilingual e salva como output para uso offline.**

- Modelo: `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`
- Sentence Transformer multilingual (inclui português)
- Tamanho: ~470 MB

---
## ⚠️ IMPORTANTE: INTERNET DEVE ESTAR **ON**

**CONFIGURACAO KAGGLE:**
1. Settings → Internet → **ON** ← **OBRIGATÓRIO para download!**
2. Settings → Accelerator → **None** (não precisa GPU)
3. Execute **Run All**
4. Clique em **Save Version** → **Save & Run All**

**Se der erro de conexão:**
- Verifique se Internet está **ON** em Settings
- Tente novamente em alguns minutos (Kaggle pode ter instabilidade temporária)

**Depois de salvar:**
- O modelo fica disponível em **Add Input → Your Work → este notebook**
- Use em outros notebooks com Internet **OFF**

---

In [ ]:
# Instalar sentence-transformers
!pip install -q sentence-transformers

# Verificar conectividade
import urllib.request
import socket

def check_internet():
    try:
        socket.setdefaulttimeout(10)
        urllib.request.urlopen('https://huggingface.co', timeout=10)
        return True
    except:
        return False

if check_internet():
    print("✅ Internet OK - conectado ao HuggingFace")
else:
    print("❌ ERRO: Sem conexão com internet!")
    print("   → Vá em Settings → Internet → ON")
    print("   → Reinicie a sessão e tente novamente")
    raise ConnectionError("Internet não está habilitada. Ative em Settings → Internet → ON")

In [ ]:
import os
from sentence_transformers import SentenceTransformer

# Modelo a baixar
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
OUTPUT_DIR = "/kaggle/working/sbert-multilingual"

print(f"Baixando modelo: {MODEL_NAME}")
print(f"Destino: {OUTPUT_DIR}")

In [ ]:
# Baixar modelo (com retry)
import time

def download_with_retry(func, *args, max_retries=3, **kwargs):
    """Tenta baixar com retry em caso de falha de rede."""
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 10
                print(f"   ⚠️ Erro na tentativa {attempt + 1}: {str(e)[:100]}")
                print(f"   Aguardando {wait_time}s antes de tentar novamente...")
                time.sleep(wait_time)
            else:
                raise

print("[1/2] Baixando Sentence-BERT...")
model = download_with_retry(SentenceTransformer, MODEL_NAME)
print("   ✅ Modelo OK")

print("[2/2] Salvando em disco...")
os.makedirs(OUTPUT_DIR, exist_ok=True)
model.save(OUTPUT_DIR)

print(f"\n✅ Modelo salvo em: {OUTPUT_DIR}")

In [ ]:
# Verificar arquivos salvos
print("Arquivos salvos:")
print("-" * 50)

total_size = 0
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        path = os.path.join(root, f)
        size = os.path.getsize(path)
        total_size += size
        rel_path = os.path.relpath(path, OUTPUT_DIR)
        print(f"  {rel_path}: {size / 1024 / 1024:.1f} MB")

print("-" * 50)
print(f"Total: {total_size / 1024 / 1024:.1f} MB")

In [ ]:
# Teste rápido - carregar modelo salvo
print("Testando carregamento do modelo salvo...")
model_test = SentenceTransformer(OUTPUT_DIR)

# Testar encoding
test_sentences = [
    "Laudo de mamografia normal",
    "Nódulo suspeito identificado",
    "BI-RADS categoria 4"
]

embeddings = model_test.encode(test_sentences)
print(f"   Shape dos embeddings: {embeddings.shape}")
print(f"   Dimensão: {embeddings.shape[1]}")

print("\n✅ Teste OK! Modelo pronto para uso offline.")

---

## ✅ Próximos Passos

1. **Save Version** → **Save & Run All (Commit)**
2. Aguarde finalizar (~5-10 min)
3. No notebook de submit, adicione este notebook como Input:
   - Add Input → Your Work → **download-sbert**
4. Use o modelo com:

```python
from sentence_transformers import SentenceTransformer

# Buscar path do modelo
MODEL_PATH = "/kaggle/input/download-sbert/sbert-multilingual"
model = SentenceTransformer(MODEL_PATH)

# Gerar embeddings
embeddings = model.encode(texts)
```